## 验证数据集  

之前我们会对数据集做train_test_split操作，将数据集分为两份    

然后进行训练，使用测试数据的误差来评判模型的好坏，如果一次训练过后，模型的在测试集上的误差较大，就会调整超参数    
直到调整到模型在测试数据集上有较好的性能，**但其实这样会使得模型在测试集上发生过拟合**   

所以引入验证数据集，我们在数据集切分时，将数据集分为三份   

然后使用训练集训练模型，使用验证集调整超参数，将测试集作为一个模型从未见过的一个模拟真实环境(生产环境)的数据集   

## 验证数据集的意义

* 验证数据集意义   
    调整超参数使用的数据集   
   
* 测试数据集意义  
    作为衡量最终模型性能的数据集   

## 交叉验证


In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

### 先使用train_test_split的方法来调超参数

In [3]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
# 使用kNN算法来进行测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [4]:
# 寻找KNN算法中的两个超参数的较优值，K，计算当前样本点与周围k个点的距离，p为距离超参数
best_score, best_k, best_p = 0, 0, 0

for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_k = k
            best_p = p
            
print("best_score", best_score)
print("best_k", best_k)
print("best_p", best_p)

best_score 0.9860917941585535
best_k 3
best_p 4


### 使用交叉验证

![crossvalidation](./img/crossvalidation.png)

* 首先将数据集分为训练集和测试集   
* 对训练集均分成k份，上图中共分为了3份    
* 对k份数据集进行组合，比如两份做训练集，剩下一份做验证数据集   
* 这样就可以形成k种训练集和验证集的组合，每种组合都能训练出一个模型   
* 我们对这k种模型得到的结果取均值来作为当前超参数发挥效果的得分（这里面使用k个模型的均值其实已经对异常值进行了处理）    
* 如果当前这k种模型的得分并不好，就调节超参数继续进行上述过程 

In [7]:
from sklearn.model_selection import cross_val_score 
import warnings 

warnings.filterwarnings("ignore")

# 创建一个默认参数的Knn分类器
knn_clf = KNeighborsClassifier()
# 直接将要进行交叉验证的分类器传入，然后传入当前的训练集
cross_val_score(knn_clf, X_train, y_train)
# 输出是三个得分，说明在当前默认超参数的情况下，对训练数据集进行了3次切分，分成了3份，然后对3个模型分别进行了计算

array([0.98895028, 0.97777778, 0.96629213])

In [8]:
# 接下来对knn模型使用交叉验证的方式寻找最优超参数k和p
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        # 将当前超参数k和p创建的knn分类器传入交叉验证函数中
        # 并将三个模型的得分进行记录
        scores = cross_val_score(knn_clf, X_train, y_train)
        # 计算三个模型得分的均值
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_p = p
            best_k = k
            
print("best_score", best_score)
print("best_k", best_k)
print("best_p", best_p)

best_score 0.9823599874006478
best_k 2
best_p 2


In [9]:
# 通过上面的交叉验证得到的最优的k和p，下面使用这两个最优超参数训练新的模型  
knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)
# 使用训练数据集进行训练
knn_clf.fit(X_train, y_train)
# 使用测试集进行模型评价，这个测试集模型是没有见过的
knn_clf.score(X_test, y_test)

0.980528511821975

## 回顾网格搜索

In [15]:
# 网格搜索原理就是使用交叉验证
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights' : ['distance'],
        'n_neighbors': [i for i in range(2, 11)],
        'p' : [i for i in range(1, 6)]
    }
]

# 创建网格搜索对象，将要搜索的分类器传入，传入相应要搜索的超参数
# verbose 日志冗长度0为不输出训练过程，1为偶尔输出
# cv参数表示将训练集分为几份，sklearn会自行计算一个合适的k值，当然我们也可以自己指定

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=3)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  1.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='distance'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

* 上面的训练过程可以解释为网格搜索首先将训练集分为3份   
* k超参数为2到11，一共有9种选择，p超参数为1到6，有5种选择，所以5x9=45种超参数组合   
* 每种模型都要分成三份进行训练和验证，一共有45x3=135种可能   

In [12]:
grid_search.best_score_

0.9823747680890538

In [13]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [14]:
# 然后我们可以直接使用网格搜索对象获取当前最优的模型
best_knn_clf = grid_search.best_estimator_ 
best_knn_clf.score(X_test, y_test)

0.980528511821975

## 留一法 Loo-CV

假设训练数据集一共有m个样本，然后这里将每个样本分为一份，就可以将数据集分为m份    
每次训练时，使用m-1个样本进行训练，然后计算该模型在最后一份样本上的得分，这样就可以训练m个模型    
将m个模型得分的均值作为衡量当前模型的好坏，进而去调节超参数  

这种方法完全不受随机的影响（如果我们使用k折交叉验证，在将训练数据集进行划分时就有很多的随机性在里面）   
可以得到最接近mo z